### Задание 1. О переговорках

Сперва сделаем допущение, что истиннй номер комнаты задается в программе до этого. Сравнивается истинный и предполагаемый номера. Если они совпали в потоки прерываются (используем библеотеку threading) и логируется (для этого используем библиотеку logging), что он угадал. Все реализованно согласно SOLID

In [11]:
import logging
from threading import Thread, Event

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

class Programmer:
    def __init__(self, name, guess_number, true_number, notification):
        self.name = name
        self.guess_number = guess_number
        self.true_number = true_number
        self.notification = notification
    
    def guess_room(self):
        if self.guess_number == self.true_number:
            logging.info(f"{self.name} угадали номер комнаты: {self.guess_number}")
            self.notification.set()

def main():
    try:
        seated_programmer_name = input("имя: ")
        guess_room_number = int(input("номер комнаты"))

        true_room_number = 3  # Предполагаемый истинный номер комнаты
        victory_notification = Event()
        
        programmer = Programmer(seated_programmer_name, guess_room_number, true_room_number, victory_notification)
        
        thread = Thread(target=programmer.guess_room)
        thread.start()
        
        victory_notification.wait()
        thread.join()
        
    except EOFError:
        logging.error("Unexpected EOF while reading input.")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


2024-03-24 13:03:44,171 - igor guessed their room number: 3


In [13]:
!pip install pyinstaller
!pyinstaller --onefile olimp1.py


Defaulting to user installation because normal site-packages is not writeable


118 INFO: PyInstaller: 6.3.0
118 INFO: Python: 3.10.12
119 INFO: Platform: Linux-6.5.0-25-generic-x86_64-with-glibc2.35
119 INFO: wrote /home/itech/Desktop/University/Я профи/olimp1.spec
script '/home/itech/Desktop/University/Я профи/olimp1.py' not found


### без отступлений от условия


In [10]:
import random
import logging
from threading import Thread, Event

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

class Programmer:
    def __init__(self, name):
        self.name = name
    
    def guess_room(self, room_number, notification):
        if room_number == self.name:
            logging.info(f"{self.name} угадали номер комнаты: {self.name}")
            notification.set()

def play_game(programmer, room_number, notification):
    programmer.guess_room(room_number, notification)

def main():
    try:
        seated_programmer_name = input("введите ваше имя ")
        
        names = []
        n = int(input("введите количество комнат"))
        
        for _ in range(n - 1):
            name, _ = input().split()
            names.append(name)
        
        room_number = ''
        for idx, name in enumerate(names, 1):
            if name == seated_programmer_name:
                room_number = str(idx)
                break
        
        programmers = [Programmer(name) for name in names]
        
        victory_notification = Event()
        
        threads = [Thread(target=play_game, args=(programmer, room_number, victory_notification)) for programmer in programmers]
        
        for t in threads:
            t.start()
        
        victory_notification.wait()
        for t in threads:
            t.join()
    
    except EOFError:
        logging.error("Unexpected EOF while reading input.")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


KeyboardInterrupt: 


## Задание 2. Поиск предмета (хвостовки) 

Начать можно с простого деления "пополам".
Начинаем с того, что 
Делим 80 сундуков на две группы по 40. Один студент проверяет каждую группу.
Если не нашли, то делим оставшиеся сундуку еще раз "пополам" в посиске золотой хвостовки.
    ...
повторяем действие.
Таким образом нам потребуется всего 7 студентов.

Для оптимизации данного решения можно воспользоваться алгоритмом бинарного поиска. Вместо того чтобы каждый раз делить все сундуки на две равные группы, мы можем использовать бинарный поиск для определения, в какой половине сундуков находится 'Золотая хвостовка'. Таким образом, мы сможем сократить количество шагов, необходимых для нахождения предмета.

Алгоритм с использованием бинарного поиска:

1. Разделим сундуки на две группы, количество сундуков в каждой группе будет равно половине общего числа сундуков.
2. Один студент проверяет сундуки в первой группе.
3. Если 'Золотая хвостовка' найдена в первой половине, процедура заканчивается.
4. Иначе, далее остаются только сундуки, которые не были проверены. Продолжаем делить оставшиеся сундуки на две равные части.
5. Следующий студент берет на себя проверку сундуков в одной из новых групп.
6. Повторяем шаги 3-5, пока 'Золотая хвостовка' не будет найдена.
Сложность нашего алгоритма (бинарного посика) O(log_{2}(n))
При n равном 80 получаем 6.32, что округляется в большую сторону до 7, тк не может быть дробной итерации.

С использованием бинарного поиска нам потребуется всего 7 студентов для гарантированного нахождения 'Золотой хвостовки' вовремя. 

In [5]:
class ChestSearch:
    def __init__(self, n):
        self.total_chests = n
        self.found = False
    
    def binary_search_tail(self, start, end):
        mid = (start + end) // 2
        if mid == end:
            self.found = True
            return mid
        return mid
    
    def find_golden_tail(self):
        start = 1
        end = self.total_chests
        step = 1
        while not self.found:
            chest = self.binary_search_tail(start, end)
            print(f"Испытатель {step} проверяет сундук с номером {chest}")
            if self.found:
                print("Хвостовка найдена!")
                return step
            else:
                if chest % 2 == 0:
                    start = chest + 1
                else:
                    end = chest - 1
            step += 1
        return step

# Создаем объект
chests = 80
search = ChestSearch(chests)

# Находим 'Золотую хвостовку'
result = search.find_golden_tail()
print(f"Итог {result}")


Испытатель 1 проверяет сундук с номером 40
Испытатель 2 проверяет сундук с номером 60
Испытатель 3 проверяет сундук с номером 70
Испытатель 4 проверяет сундук с номером 75
Испытатель 5 проверяет сундук с номером 72
Испытатель 6 проверяет сундук с номером 73
Испытатель 7 проверяет сундук с номером 72
Хвостовка найдена!
Итог 7


Теперь добавим возможность пользователю выпольнить поиск 

In [3]:
class ChestSearchAlgorithm:
    def __init__(self):
        self.students = 0
        self.attempts = 0
        self.chests_checked = 0

    def binary_search_tail(self, start, end, group_num):
        middle = (start + end) // 2
        self.attempts += 1
        group_size = end - start
        self.chests_checked += group_size

        tail_found = input(f"Студент {group_num} нашел ли 'Золотую хвостовку' в сундуке с номером {middle}? (да/нет): ")

        if tail_found.lower() == 'да':
            return True
        else:
            return False

    def find_minimum_students(self):
        success = False
        while not success:
            self.students += 1
            self.attempts = 0
            self.chests_checked = 0

            group = 1
            success = self.binary_search_tail(1, 81, group)

            while not success and self.chests_checked < 80:
                group += 1
                start = 1 + (group - 1) * 40
                end = min(group * 40, 81)

                success = self.binary_search_tail(start, end, group)

            print(f"Студентов: {self.students}, Проверено сундуков: {self.chests_checked}")

        return self.students

def main():
    search_algorithm = ChestSearchAlgorithm()
    min_students = search_algorithm.find_minimum_students()

    print(f"Минимальное количество испытателей: {min_students}")

if __name__ == "__main__":
    main()


Студентов: 1, Проверено сундуков: 80
Студентов: 2, Проверено сундуков: 80
Студентов: 3, Проверено сундуков: 80
Студентов: 4, Проверено сундуков: 80
Студентов: 5, Проверено сундуков: 80
Студентов: 6, Проверено сундуков: 80
Студентов: 7, Проверено сундуков: 80
Студентов: 8, Проверено сундуков: 80
Минимальное количество испытателей: 8


## Задание 3. Умный фильтр пользователей


### Алгоритм, который можно использовать для поиска пользователей в базе данных (БД):

1. Подключение к базе данных: Сначала устанавливается соединение с базой данных. Подключение может быть установлено с помощью библиотеки или API для работы с базой данных, такими как sqlite3 для SQLite

2. Выбор стратегии поиска: Определяем, какой тип поиска мы хотим выполнить: по имени, фамилии, полному имени или email. Создаем соответствующий объект стратегии поиска (например, FirstNameSearch, LastNameSearch, FullNameSearch, EmailSearch).

3. Формирование SQL-запроса и выполнение запроса: Используем данные запроса, при необходимости выполняем преобразования, например, изменение на lower_case для игнорирования регистра.

4. Получение результатов поиска с БД

5. Отображение результатов и закрытие соединения с БД



### Подключение к бд и ее запонение тестовыми данными

In [1]:
import sqlite3

# Создаем и заполняем таблицу пользователей и выполним подключение к БД
conn = sqlite3.connect('users.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY,
                    first_name TEXT,
                    last_name TEXT,
                    middle_name TEXT,
                    email TEXT
                  )''')

users = [
    ('Иван', 'Иванов', 'Иванович', 'ivanov@example.com'),
    ('Петр', 'Петров', 'Петрович', 'petrov@example.com'),
    ('Мария', 'Сидорова', 'Петровна', 'sidorova@example.com'),
    ('Анна', 'Кузнецова', 'Александровна', 'kuznetsova@example.com')
]

cursor.executemany('INSERT INTO users (first_name, last_name, middle_name, email) VALUES (?, ?, ?, ?)', users)

conn.commit()
conn.close()

# Классы для поиска пользователей
# По имени
class FirstNameSearch:
    def search(self, cursor, query):
        cursor.execute("SELECT * FROM users WHERE first_name LIKE ?", ('%' + query + '%',))
        return cursor.fetchall()

# По фамилии
class LastNameSearch:
    def search(self, cursor, query):
        cursor.execute("SELECT * FROM users WHERE last_name LIKE ?", ('%' + query + '%',))
        return cursor.fetchall()

# По отчеству
class FullNameSearch:
    def search(self, cursor, query):
        cursor.execute("SELECT * FROM users WHERE first_name || ' ' || last_name || ' ' || middle_name LIKE ?", ('%' + query + '%',))
        return cursor.fetchall()

# по электронной почте
class EmailSearch:
    def search(self, cursor, query):
        cursor.execute("SELECT * FROM users WHERE email LIKE ?", ('%' + query + '%',))
        return cursor.fetchall()

class UserSearcher:
    def search_users(self, strategy, query):
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()

        results = strategy.search(cursor, query)

        conn.close()

        return results



### Пример использования поиска по всем параметрам
Мы рассматриваем все возможные статегии поиска [FirstNameSearch(), LastNameSearch() ...], представленные в классе стратегии пользователя (стараемся придерживаться SOLID)

Поместите свой запрос в search_query для поиска в БД

In [64]:

searcher = UserSearcher()

search_query = 'Иван Иванов Иванович'
queries_list = search_query.split()  # Разбиваем строку на список запросов

for query in queries_list:
    print(f"Результаты по запросу '{query}':")
    for strategy in [FirstNameSearch(), LastNameSearch(), FullNameSearch(), EmailSearch()]:
        results = searcher.search_users(strategy, query)
        if results:
            for user in results:
                print(user)
            
        else:
            print("Пользователи не найдено .")

Результаты по запросу 'Иван':
(1, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(5, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(9, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(13, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(1, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(5, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(9, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(13, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(1, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(5, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(9, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(13, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
Пользователи не найдено .
Результаты по запросу 'Иванов':
Пользователи не найдено .
(1, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(5, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(9, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(13, 'Иван', 'Иванов', 'Иванович', 'ivanov@

In [61]:
# Класс для поиска по полному имени
class FullSearch:
    def search(self, cursor, query, ignore_case=False):
        if ignore_case:
            cursor.execute("""SELECT * FROM users 
                              WHERE lower(first_name || ' ' || last_name || ' ' || middle_name || email) LIKE ?""", ('%' + query.lower() + '%',))
        else:
            cursor.execute("""SELECT * FROM users 
                              WHERE first_name || ' ' || last_name || ' ' || middle_name || email LIKE ?""", ('%' + query + '%',))
        return cursor.fetchall()

#  для учета регистра добавим ignore_case в UserSearcher
class UserSearcher:
    def search_users(self, strategy, query,ignore_case):
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()

        results = strategy.search(cursor, query)

        conn.close()

        return results

# Пример использования с возможностью игнорирования регистра
searcher = UserSearcher()

search_query = "ivanov@example.com"
ignore_case = False  # Игнорировать регистр

print(f"Результаты по запросу '{search_query}':")
for strategy in [FullSearch()]:
    results = searcher.search_users(strategy, search_query,ignore_case)
    if results:
        for user in results:
            print(user)
    else:
        print("Пользователи не найдены.")


Результаты по запросу 'ivanov@example.com':
(1, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(5, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')
(9, 'Иван', 'Иванов', 'Иванович', 'ivanov@example.com')


## Решения демоверсии также опубликованно на github:

 1. https://github.com/ITech7750/Profi_SE

 2. https://github.com/ITech7750/spring-jpa-rest-api

 3. https://github.com/ITech7750/channel-ratings-predictor